In [1]:
! pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 462 kB 26.6 MB/s 


In [2]:
! pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import os
import zipfile
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from glob import glob #pour extrraire le nombre de fichier / extraire le nombre de classes
from tqdm import tqdm_notebook 
from keras.preprocessing.image import ImageDataGenerator #preparation de données avec a fonction pour augmenter les données (zoom, rotation)
from keras.layers import Input, Lambda, Dense, Flatten #l'idée de transfer learning on a le reseau entrainé mais il faut rajouter des couche a la sortie  pour le personnaliser a notre utilisation
from keras.models import load_model, Model
from keras.applications.vgg16 import VGG16 
from keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications import EfficientNetB0
from google.colab import drive
drive.mount('/content/drive')
%matplotlib inline
tf.__version__


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'2.8.0'

In [4]:
#importer la base de données
data_path='/content/drive/My Drive/Image_data/Pictures-classification-2'

In [5]:
#spécifier les chemins de la base de données train et test 
train_dir = '/content/drive/MyDrive/DataSet/Pictures-classification-2/train'
test_dir = '/content/drive/MyDrive/DataSet/Pictures-classification-2/test'

In [6]:
#création du reseau 
#spécifier la taille d'image
IMG_SHAPE = (224, 224,3)

In [7]:
# preparer le model de base : le vgg16 deja entrainé 
base_model = EfficientNetB0(input_shape=IMG_SHAPE, include_top=False, weights='imagenet') #include : rajouter d'autre couche a la sortie 

16719872/16705208 [==============================] - 0s 0us/step


In [8]:
#presenter la structure de notre reseau
base_model.summary()

Model: "efficientnetb0"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 224, 224, 3)  0           ['input_1[0][0]']                
                                                                                                  
 normalization (Normalization)  (None, 224, 224, 3)  7           ['rescaling[0][0]']              
                                                                                                  
 stem_conv_pad (ZeroPadding2D)  (None, 225, 225, 3)  0           ['normalization[0][0

In [9]:
#on doit bloquer le model de base : une fois je lance l'entrainement les parametres ne vont pas changer 
base_model.trainable = False

In [10]:
#pour voir la taille de sorite 512 on veut 2 seulement 
base_model.output

<KerasTensor: shape=(None, 7, 7, 1280) dtype=float32 (created by layer 'top_activation')>

In [11]:
# useful for getting number of classes
folders = glob(train_dir+'/*')

In [12]:
folders

['/content/drive/MyDrive/DataSet/Pictures-classification-2/train/NAD',
 '/content/drive/MyDrive/DataSet/Pictures-classification-2/train/AD']

In [13]:
# ajouter les couches de sorties 
x = Flatten()(base_model.output) # flatten pour que la matrice de sortie de mon reseau  soit une matrice lineaire 
# x = Dense(1000, activation='relu')(x)
prediction = Dense(len(folders), activation='softmax')(x)

In [14]:
#creation le modele global : tout on a fait le lien entre le petit reseau et le grand reseau 
model = Model(inputs=base_model.input, outputs=prediction)

In [15]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 224, 224, 3)  0           ['input_1[0][0]']                
                                                                                                  
 normalization (Normalization)  (None, 224, 224, 3)  7           ['rescaling[0][0]']              
                                                                                                  
 stem_conv_pad (ZeroPadding2D)  (None, 225, 225, 3)  0           ['normalization[0][0]']      

In [16]:
#compiler le reseau 
model.compile(loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy'])
#fonction loss qui calcule la perte c'est comme on a une variable categoriqie 

In [17]:
#créer les photos augmenter 
train_datagen = ImageDataGenerator(rescale = 1./255,       
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255) #juste standariser les photos 
#parcourir les fichier et appliqueer train_test_datagen
training_set = train_datagen.flow_from_directory(train_dir,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory(test_dir,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 3163 images belonging to 2 classes.
Found 68 images belonging to 2 classes.


In [18]:
import os
dir = os.listdir('/content/drive/MyDrive/DataSet/Pictures-classification-2/train')
print(dir)

['NAD', 'AD']


In [19]:
ls

drive/  sample_data/


In [20]:
#appliquer le model : faire passer les données dans mon reseau 
model.fit_generator(training_set, epochs=4, validation_data=test_set)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/4
99/99 [==============================] - 546s 5s/step - loss: 2.1085 - accuracy: 0.5122 - val_loss: 0.9168 - val_accuracy: 0.3971
Epoch 2/4
99/99 [==============================] - 169s 2s/step - loss: 0.9522 - accuracy: 0.5194 - val_loss: 1.1895 - val_accuracy: 0.3971
Epoch 3/4
99/99 [==============================] - 169s 2s/step - loss: 0.8682 - accuracy: 0.5229 - val_loss: 0.6920 - val_accuracy: 0.6029
Epoch 4/4
99/99 [==============================] - 171s 2s/step - loss: 0.9565 - accuracy: 0.5261 - val_loss: 1.1474 - val_accuracy: 0.6029


In [21]:
#pour connaitre la precision 
valid_loss, valid_accuracy = model.evaluate_generator(test_set)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  


In [22]:
#precision :
print("Accuracy after transfer learning: {}".format(valid_accuracy)) 

Accuracy after transfer learning: 0.6029411554336548


In [23]:
import pickle
pickle.dump(model, open('model.pkl', 'wb'))

INFO:tensorflow:Assets written to: ram://23b7ab35-4db7-4973-ae99-71defa776a22/assets


In [24]:
!pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.1 MB 23.7 MB/s 
     |████████████████████████████████| 54 kB 2.1 MB/s 
     |████████████████████████████████| 253 kB 67.2 MB/s 
     |████████████████████████████████| 84 kB 3.0 MB/s 
     |████████████████████████████████| 212 kB 61.0 MB/s 
     |████████████████████████████████| 53 kB 2.0 MB/s 
     |████████████████████████████████| 2.0 MB 39.3 MB/s 
     |████████████████████████████████| 1.1 MB 41.9 MB/s 
     |████████████████████████████████| 144 kB 70.9 MB/s 
     |████████████████████████████████| 94 kB 2.9 MB/s 
     |████████████████████████████████| 271 kB 51.4 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
     |████████████████████████████████| 11.1 MB 55.7 MB/s 
     |████████████████████████████████| 80 kB 8.3 MB/s 
     |████████████████████████████████| 43 kB 1.7 MB/s 
     |████████████████████████████████| 856 

In [25]:
def classify_image(inp):
  inp = inp.reshape((-1, 224, 224, 3))
  prediction = model.predict(inp).flatten()
  # labels = ["AD", "NAD"]
  confidences = {"NAD": float(prediction[0]),"AD": 1- float(prediction[0])}
  # print(confidences)
  return confidences



In [26]:
import gradio as gr

gr.Interface(fn=classify_image, 
             inputs=gr.inputs.Image(shape=(224, 224)),
             outputs=gr.outputs.Label(num_top_classes=1)).launch(debug=True)

/usr/local/lib/python3.7/dist-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.7/dist-packages/gradio/deprecation.py:40: UserWarning: The 'type' parameter has been deprecated. Use the Number component instead.
  warnings.warn(value)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://45664.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.


(<gradio.routes.App at 0x7f91de396510>,
 'http://127.0.0.1:7860/',
 'https://45664.gradio.app')